In [9]:
import pandas as pd
import datetime 

In [10]:
df = pd.read_csv('../data/parsed_logs_rad_may_2024.csv')

In [11]:
grouped_df = df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})

In [12]:
grouped_df["SDC_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]
grouped_df["Critical_rate"] = grouped_df["critical_sdc"] / grouped_df["acc_time"]

In [13]:
grouped_df.loc[pd.IndexSlice[:, "ON", :]]

batchsize  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision              
carola20001 ON  swin_base_patch4_window12_384 0.00010        None      vits           False      fp32              32   
                                                                                      True       fp32              32   
                                              0.01000        None      vits           False      fp32              32   
                swin_base_patch4_window7_224  0.00001        None      vits           False      fp16              32   
                                                                                                 fp32              32   
                                                                                      True       fp16              32   
                                                                                                 fp32              32   
                vit_base_patch16_384          0.00001        None      vits           False      fp16              32   
                                                                                      True       fp16              32   
carola20002 ON  groundingdino_swinb_cogcoor   0.00001        Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                groundingdino_swint_ogc       0.00001        Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   

                                                                                                                acc_time  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision                 
carola20001 ON  swin_base_patch4_window12_384 0.00010        None      vits           False      fp32        2383.258363   
                                                                                      True       fp32         589.985342   
                                              0.01000        None      vits           False      fp32         115.231299   
                swin_base_patch4_window7_224  0.00001        None      vits           False      fp16        5764.940648   
                                                                                                 fp32        6617.565160   
                                                                                      True       fp16        6022.334731   
                                                                                                 fp32        1554.594530   
                vit_base_patch16_384          0.00001        None      vits           False      fp16        4365.247321   
                                                                                      True       fp16        2222.712865   
carola20002 ON  groundingdino_swinb_cogcoor   0.00001        Attention grounding_dino False      fp32       14171.472841   
                                                                                      True       fp32       10595.112528   
                groundingdino_swint_ogc       0.00001        Attention grounding_dino False      fp32        9066.553506   
                                                                                      True       fp32        7050.240353   

                                                                                                            sdc  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision        
carola20001 ON  swin_base_patch4_window12_384 0.00010        None      vits           False      fp32        43   
                                                                                      

In [14]:
ft_df = df[pd.to_datetime(df["start_dt"]) > datetime.datetime(year=2024, month=5, day=24, hour=12)]
new_group = ft_df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})

In [17]:
new_group["SDC_rate"] = new_group["sdc"] / new_group["acc_time"]
new_group["Critical_rate"] = new_group["critical_sdc"] / new_group["acc_time"]

In [18]:
new_group

batchsize  \
hostname    ecc model                        floatthreshold microop   setup_type     hardenedid precision              
carola20001 ON  swin_base_patch4_window7_224 0.00001        None      vits           False      fp32              32   
                                                                                     True       fp32              32   
                vit_base_patch16_384         0.00001        None      vits           False      fp16              32   
                                                                                     True       fp16              32   
carola20002 ON  groundingdino_swinb_cogcoor  0.00001        Attention grounding_dino False      fp32               1   
                                                                                     True       fp32               1   
                groundingdino_swint_ogc      0.00001        Attention grounding_dino False      fp32               1   
                                                                                     True       fp32               1   
carolp22003 OFF SNN_NMNIST                   0.00000        0         0              0          0                  0   

                                                                                                               acc_time  \
hostname    ecc model                        floatthreshold microop   setup_type     hardenedid precision                 
carola20001 ON  swin_base_patch4_window7_224 0.00001        None      vits           False      fp32        5636.983286   
                                                                                     True       fp32        1191.388981   
                vit_base_patch16_384         0.00001        None      vits           False      fp16        4365.247321   
                                                                                     True       fp16        2222.712865   
carola20002 ON  groundingdino_swinb_cogcoor  0.00001        Attention grounding_dino False      fp32        7047.010326   
                                                                                     True       fp32        4144.507730   
                groundingdino_swint_ogc      0.00001        Attention grounding_dino False      fp32        6362.109609   
                                                                                     True       fp32        5765.341707   
carolp22003 OFF SNN_NMNIST                   0.00000        0         0              0          0          13813.099881   

                                                                                                            sdc  \
hostname    ecc model                        floatthreshold microop   setup_type     hardenedid precision         
carola20001 ON  swin_base_patch4_window7_224 0.00001        None      vits           False      fp32         52   
                                                                                     True       fp32         25   
                vit_base_patch16_384         0.00001        None      vits           False      fp16        157   
                                                                                     True       fp16         58   
carola20002 ON  groundingdino_swinb_cogcoor  0.00001        Attention grounding_dino False      fp32        107   
                                                                                     True       fp32         80   
                groundingdino_swint_ogc      0.00001        Attention grounding_dino False      fp32        190   
                                                                                     True       fp32        136   
carolp22003 OFF SNN_NMNIST                   0.00000        0         0              0          0          5301   

                                                                                                           critical_sdc  \
hostname    ecc model                      

In [16]:
grouped_df.loc[pd.IndexSlice[:, "OFF", :]]


batchsize  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01           None      vits           False      fp32               8   
                vit_base_patch16_224          0.01           None      vits           False      fp32               8   
                                                                                      True       fp32               8   
                                              0.10           None      vits           False      fp16               8   
                                                                                      True       fp16               8   
                vit_base_patch16_384          0.01           Attention microbenchmark False      fp32               8   
                                                             Block     microbenchmark False      fp32               8   
carolp20002 OFF groundingdino_swinb_cogcoor   0.01           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                                              0.10           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                groundingdino_swint_ogc       0.01           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
carolp22003 OFF SNN_NMNIST                    0.00           0         0              0          0                  0   

                                                                                                                acc_time  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision                 
carola20001 OFF swin_base_patch4_window12_384 0.01           None      vits           False      fp32         617.650461   
                vit_base_patch16_224          0.01           None      vits           False      fp32        1899.688553   
                                                                                      True       fp32        1463.919021   
                                              0.10           None      vits           False      fp16        1669.635372   
                                                                                      True       fp16         196.248820   
                vit_base_patch16_384          0.01           Attention microbenchmark False      fp32        1195.039556   
                                                             Block     microbenchmark False      fp32        2698.228651   
carolp20002 OFF groundingdino_swinb_cogcoor   0.01           Attention grounding_dino False      fp32        3923.360195   
                                                                                      True       fp32        1427.972256   
                                              0.10           Attention grounding_dino False      fp32        5554.948415   
                                                                                      True       fp32         642.790403   
                groundingdino_swint_ogc       0.01           Attention grounding_dino False      fp32        9353.682741   
                                                                                      True       fp32        3337.683801   
carolp22003 OFF SNN_NMNIST                    0.00           0         0              0          0          26212.332484   

                                                                                                             sdc  \
hostname    ecc model                         floatthreshold microop  